<a href="https://colab.research.google.com/github/Lexi-Zhou/stats201-project-zzz/blob/main/Code/W4_1_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Load data

In [ ]:
import re
import os
import string
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/16_RMP_gender_finalized.csv"

df = pd.read_csv(FILE_PATH)
df.head()

,professor_name,school_name,department_name_clean,stu_tags,student_star,comments,tag_list,comment_gender_signal,prof_gender_label,star_rating,...,comments_basic,comments_expanded,comments_tokens,comments_final,n_tokens_final,professor_first_name,prof_gender_ssa,prof_gender_crosschecked,prof_likely_gender,prof_gender_finalized
0,Lynn Ketter,Northwest Florida State College,Education,RESPECTED CARING GIVES GOOD FEEDBACK,5.0,Wonderful teacher. Basically all work is done ...,"['RESPECTED', 'CARING', 'GIVES GOOD FEEDBACK']",female,female,3.7,...,Wonderful teacher. Basically all work is done ...,wonderful teacher. basically all work is done ...,"['wonderful', 'teacher', 'basically', 'work', ...",wonderful teacher basically work class grading...,15,lynn,female,female,unknown,female
1,Lynn Ketter,Northwest Florida State College,Education,PARTICIPATION MATTERS,2.0,Discussion boards are mandatory. Use book for ...,['PARTICIPATION MATTERS'],female,female,3.7,...,Discussion boards are mandatory. Use book for ...,discussion boards are mandatory. use book for ...,"['discussion', 'boards', 'mandatory', 'use', '...",discussion boards mandatory use book tests per...,26,lynn,female,female,unknown,female
2,Lynn Ketter,Northwest Florida State College,Education,GIVES GOOD FEEDBACK LOTS OF HOMEWORK PARTICI...,3.5,"Honestly, I didn\'t learn anything in this cla...","['GIVES GOOD FEEDBACK', 'LOTS OF HOMEWORK', 'P...",unknown,female,3.7,...,"Honestly, I didn't learn anything in this clas...","honestly, i did not learn anything in this cla...","['honestly', 'did', 'not', 'NEG_learn', 'class...",honestly did not NEG_learn class ketter amazin...,15,lynn,female,female,unknown,female
3,Lynn Ketter,Northwest Florida State College,Education,GIVES GOOD FEEDBACK LOTS OF HOMEWORK SKIP CL...,4.0,She comes off a little cold at first but she\'...,"['GIVES GOOD FEEDBACK', 'LOTS OF HOMEWORK', ""S...",female,female,3.7,...,She comes off a little cold at first but she's...,she comes off a little cold at first but she i...,"['comes', 'little', 'cold', 'actually', 'reall...",comes little cold actually really nice lot pas...,26,lynn,female,female,unknown,female
4,Lynn Ketter,Northwest Florida State College,Education,NaN,5.0,I thought she was a great teacher and always g...,[],female,female,3.7,...,I thought she was a great teacher and always g...,i thought she was a great teacher and always g...,"['thought', 'great', 'teacher', 'gave', 'helpf...",thought great teacher gave helpful feedback ju...,10,lynn,female,female,unknown,female


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19687 entries, 0 to 19686
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   professor_name            19687 non-null  object 
 1   school_name               19687 non-null  object 
 2   department_name_clean     19687 non-null  object 
 3   stu_tags                  4975 non-null   object 
 4   student_star              19674 non-null  float64
 5   comments                  19586 non-null  object 
 6   tag_list                  19687 non-null  object 
 7   comment_gender_signal     19687 non-null  object 
 8   prof_gender_label         19687 non-null  object 
 9   star_rating               19687 non-null  float64
 10  star_rating_category      19687 non-null  object 
 11  student_star_category     19674 non-null  object 
 12  comments_raw              19586 non-null  object 
 13  comments_basic            19586 non-null  object 
 14  commen

# 2. Comments

## 2.1 Punctuation Removal

In [ ]:
# Convert NaN to an empty string, and convert to str
df['comments_expanded'] = df['comments_expanded'].fillna('').astype(str)

punct_pattern = r"[{}]".format(re.escape(string.punctuation))

df['comments_expanded'] = df['comments_expanded'].apply(
    lambda x: re.sub(punct_pattern, '', x)
)

df.head()

,professor_name,school_name,department_name_clean,stu_tags,student_star,comments,tag_list,comment_gender_signal,prof_gender_label,star_rating,...,comments_basic,comments_expanded,comments_tokens,comments_final,n_tokens_final,professor_first_name,prof_gender_ssa,prof_gender_crosschecked,prof_likely_gender,prof_gender_finalized
0,Lynn Ketter,Northwest Florida State College,Education,RESPECTED CARING GIVES GOOD FEEDBACK,5.0,Wonderful teacher. Basically all work is done ...,"['RESPECTED', 'CARING', 'GIVES GOOD FEEDBACK']",female,female,3.7,...,Wonderful teacher. Basically all work is done ...,wonderful teacher basically all work is done i...,"['wonderful', 'teacher', 'basically', 'work', ...",wonderful teacher basically work class grading...,15,lynn,female,female,unknown,female
1,Lynn Ketter,Northwest Florida State College,Education,PARTICIPATION MATTERS,2.0,Discussion boards are mandatory. Use book for ...,['PARTICIPATION MATTERS'],female,female,3.7,...,Discussion boards are mandatory. Use book for ...,discussion boards are mandatory use book for t...,"['discussion', 'boards', 'mandatory', 'use', '...",discussion boards mandatory use book tests per...,26,lynn,female,female,unknown,female
2,Lynn Ketter,Northwest Florida State College,Education,GIVES GOOD FEEDBACK LOTS OF HOMEWORK PARTICI...,3.5,"Honestly, I didn\'t learn anything in this cla...","['GIVES GOOD FEEDBACK', 'LOTS OF HOMEWORK', 'P...",unknown,female,3.7,...,"Honestly, I didn't learn anything in this clas...",honestly i did not learn anything in this clas...,"['honestly', 'did', 'not', 'NEG_learn', 'class...",honestly did not NEG_learn class ketter amazin...,15,lynn,female,female,unknown,female
3,Lynn Ketter,Northwest Florida State College,Education,GIVES GOOD FEEDBACK LOTS OF HOMEWORK SKIP CL...,4.0,She comes off a little cold at first but she\'...,"['GIVES GOOD FEEDBACK', 'LOTS OF HOMEWORK', ""S...",female,female,3.7,...,She comes off a little cold at first but she's...,she comes off a little cold at first but she i...,"['comes', 'little', 'cold', 'actually', 'reall...",comes little cold actually really nice lot pas...,26,lynn,female,female,unknown,female
4,Lynn Ketter,Northwest Florida State College,Education,NaN,5.0,I thought she was a great teacher and always g...,[],female,female,3.7,...,I thought she was a great teacher and always g...,i thought she was a great teacher and always g...,"['thought', 'great', 'teacher', 'gave', 'helpf...",thought great teacher gave helpful feedback ju...,10,lynn,female,female,unknown,female


## 2.2 Stopword Removal

Remove stopwords but retain negations, intensifiers, and gendered pronouns

In [ ]:
stop_words = set(ENGLISH_STOP_WORDS)

negations = {'no', 'not', 'never', 'nor', 'without'}
intensifiers = {'very', 'really', 'too', 'so', 'extremely', 'quite', 'super'}
pronouns_keep = {'he','she','him','her','his','hers'}

custom_keep = negations | intensifiers | pronouns_keep
filtered_stopwords = stop_words - custom_keep

def remove_stopwords(text):
    tokens = text.split()
    tokens = [w for w in tokens if w not in filtered_stopwords]
    return " ".join(tokens)

df['comments_nostop'] = df['comments_expanded'].apply(remove_stopwords)

## 2.3 Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = text.split()
    lemmas = [lemmatizer.lemmatize(w, pos='v') for w in tokens]
    lemmas = [lemmatizer.lemmatize(w, pos='n') for w in lemmas]
    return " ".join(lemmas)

df['comments_lemma_original'] = df['comments_nostop'].apply(lemmatize_text)

## 2.4 Pronoun Scrubbing

Remove gendered pronouns

In [ ]:
# add 'hes','shes','man','woman','guy','lady','m'
pronouns_remove = {'he','she','him','her','his','hers','hes','shes','man','woman','guy','lady','m'}

def remove_pronouns(text):
    tokens = text.split()
    tokens = [w for w in tokens if w not in pronouns_remove]
    return " ".join(tokens)

df['comments_lemma_scrubbed'] = df['comments_lemma_original'].apply(remove_pronouns)


## 2.5 Backup Columns with Negation Marking (scope=1)

In [ ]:
NEGATION_WORDS = {"no", "not", "never", "nor", "without"}

def negation_mark(text):
    tokens = text.split()
    new_tokens = []
    i = 0
    while i < len(tokens):
        word = tokens[i]
        new_tokens.append(word)
        if word in NEGATION_WORDS and i+1 < len(tokens):
            new_tokens.append("NEG_" + tokens[i+1])
            i += 2
        else:
            i += 1
    return " ".join(new_tokens)

df['comments_lemma_original_neg'] = df['comments_lemma_original'].apply(negation_mark)
df['comments_lemma_scrubbed_neg'] = df['comments_lemma_scrubbed'].apply(negation_mark)

## 2.6 df Arrangement

Standardize column names and retain only analysis-relevant variables

In [ ]:
# Column Renaming
rename_map = {
    'comments_lemma_original': 'comments_original',
    'comments_lemma_scrubbed': 'comments_scrubbed',
    'comments_lemma_original_neg': 'comments_original_neg',
    'comments_lemma_scrubbed_neg': 'comments_scrubbed_neg'
}

df = df.rename(columns=rename_map)

In [ ]:
# Columns to Keep
columns_to_keep = [
    'professor_name',
    'prof_gender_finalized',
    'department_name_clean',
    'star_rating',
    'star_rating_category',
    'comments',
    'comments_original',      # orginal
    'comments_scrubbed',      # scrubbed
    'comments_original_neg',  # orginal+negation (backup)
    'comments_scrubbed_neg',  # scrubbed+negation (backup)
]

df = df.reindex(columns=columns_to_keep)
df.head()

,professor_name,prof_gender_finalized,department_name_clean,star_rating,star_rating_category,comments,comments_original,comments_scrubbed,comments_original_neg,comments_scrubbed_neg
0,Lynn Ketter,female,Education,3.7,good,Wonderful teacher. Basically all work is done ...,wonderful teacher basically work class grade v...,wonderful teacher basically work class grade v...,wonderful teacher basically work class grade v...,wonderful teacher basically work class grade v...
1,Lynn Ketter,female,Education,3.7,good,Discussion boards are mandatory. Use book for ...,discussion board mandatory use book test perso...,discussion board mandatory use book test perso...,discussion board mandatory use book test perso...,discussion board mandatory use book test perso...
2,Lynn Ketter,female,Education,3.7,good,"Honestly, I didn\'t learn anything in this cla...",honestly do not learn class ketter amaze give ...,honestly do not learn class ketter amaze give ...,honestly do not NEG_learn class ketter amaze g...,honestly do not NEG_learn class ketter amaze g...
3,Lynn Ketter,female,Education,3.7,good,She comes off a little cold at first but she\'...,she come little cold she actually really nice ...,come little cold actually really nice lot pass...,she come little cold she actually really nice ...,come little cold actually really nice lot pass...
4,Lynn Ketter,female,Education,3.7,good,I thought she was a great teacher and always g...,think she great teacher give helpful feedback ...,think great teacher give helpful feedback just...,think she great teacher give helpful feedback ...,think great teacher give helpful feedback just...


# 3. Department Classification (STEM vs Humanities)

In [ ]:
df['department_name_clean'].unique()

array(['Education', 'Business', 'Communication', 'Public Administration',
       'Computer Science', 'Criminal Justice', 'Sociology',
       'Physical Education', 'Art', 'History', 'Mathematics',
       'Humanities', 'English', 'Process Technology', 'Psychology',
       'Physics', 'Hospitality', 'Social Work', 'Law Enforcement',
       'Foreign Languages', 'Chemistry', 'Health Science',
       'Elementary Education', 'Biology', 'Law', 'Writing', 'Science',
       'Contract Drafting', 'Economics', 'Child & Family Studies',
       'Religion', 'Languages', 'Digital Arts', 'First Amendment',
       'Comparative Literature', 'Nursing',
       'Hotel & Restaurant Management', 'Journalism', 'Art & Design',
       'Electronics', 'Philosophy', 'American Studies', 'Fashion',
       'Finance', 'Classics', 'Nutrition', 'Environmental Studies',
       'Environmental Engineering', 'Engineering', 'Studio Art',
       'Culinary Arts', 'Music', 'Aerospace Studies', 'Anthropology',
       'Merchandising

In [ ]:
stem_keywords = [
    # hard sciences
    "science", "physics", "chemistry", "biology", "biochem", "genetic",
    "geology", "geoscience", "geophysics", "marine", "zoology", "anatomy",
    "physiology", "microbiology", "biomedical", "biological",

    # math & computing
    "math", "mathematics", "statistics", "computer", "software",
    "information", "technology", "data", "systems", "engineering",
    "electrical", "mechanical", "industrial", "civil", "chemical",
    "aerospace", "bioengineering",

    # health & medical
    "nursing", "medicine", "health", "radiological", "dental",
    "pharmacy", "exercise science", "kinesiology", "nutrition",
    "sports medicine", "public health",

    # applied tech
    "electronics", "manufacturing", "welding", "automotive",
    "air conditioning", "construction"
]

humanities_keywords = [
    # arts & culture
    "art", "design", "music", "theater", "dance", "film", "media",
    "photography", "fashion", "illustration", "studio art",

    # language & literature
    "language", "linguistics", "literature", "writing", "rhetoric",
    "english", "french", "spanish", "german", "italian", "persian",
    "russian", "latin", "scandinavian", "asian studies",

    # social sciences
    "history", "philosophy", "religion", "theology", "anthropology",
    "sociology", "psychology", "political", "government",
    "public administration", "policy", "criminal justice",
    "law", "social work", "human services", "education",
    "communication", "journalism", "cultural studies",
    "ethnic studies", "women", "family", "child development",
    "international studies", "american studies"
]

In [ ]:
def classify_department(dept):
    if pd.isna(dept):
        return np.nan

    d = dept.lower()

    # STEM first (Avoid having "health education" classified as "education")
    for k in stem_keywords:
        if k in d:
            return "STEM"

    for k in humanities_keywords:
        if k in d:
            return "Humanities"

    # fallback
    return "Humanities"


# insert after the department column
col_index = df.columns.get_loc("department_name_clean") + 1
df.insert(col_index, "department_domain", df["department_name_clean"].apply(classify_department))

# check
missing_department_label = df["department_domain"].isna()
print("Number of unlabeled departments:", missing_department_label.sum())

Number of unlabeled departments: 0


In [ ]:
OUTPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/17_RMP_comment_department.csv"
df.to_csv(OUTPUT_FILE, index=False)

print(f"\nFinal DataFrame saved to: {OUTPUT_FILE}")


Final DataFrame saved to: /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/17_RMP_comment_department.csv


# 4. Split DataFrame

This step creates 6 separate groups based on the combination of department domain (STEM/Humanities) and rating category (Poor, Average, Good).
Each group is stored as an independent DataFrame and saved as a CSV.

In [ ]:
departments = ['STEM', 'Humanities']
ratings = ['poor', 'average', 'good']

# Store each group of DataFrames in a dictionary
group_dfs = {}

for dep in departments:
    for rat in ratings:
        group_name = f"df_{dep.lower()}_{rat.lower()}"
        # copy subgroup to new DataFrame
        group_dfs[group_name] = df[(df["department_domain"] == dep) &
                                    (df["star_rating_category"] == rat)].copy()
        print(f"{group_name} created | rows: {len(group_dfs[group_name])}")

OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

for name, gdf in group_dfs.items():
    file_name = f"18_RMP_{name.split('df_')[1]}.csv"
    output_file = os.path.join(OUTPUT_DIR, file_name)
    gdf.to_csv(output_file, index=False)
    print(f"Saved {name} to {output_file}")


df_stem_poor created | rows: 1022
df_stem_average created | rows: 1581
df_stem_good created | rows: 4161
df_humanities_poor created | rows: 1101
df_humanities_average created | rows: 2429
df_humanities_good created | rows: 9393
Saved df_stem_poor to /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/18_RMP_stem_poor.csv
Saved df_stem_average to /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/18_RMP_stem_average.csv
Saved df_stem_good to /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/18_RMP_stem_good.csv
Saved df_humanities_poor to /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/18_RMP_humanities_poor.csv
Saved df_humanities_average to /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/18_RMP_humanities_average.csv
Saved df_humanities_good to /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_4/18_RMP_humanities_good.csv
